# KML do nalotów

widgety do wprowadzania danych

In [5]:
#setup UI
import ipywidgets as widgets
wLat = widgets.BoundedFloatText(
 value=50.936667,
 min=0,
 max=60,
 description='Lat:'
 )
wLon = widgets.BoundedFloatText(
 value=16.861389,
 min=0,
 max=20,
 description='Lon:'
 )
wRadius = widgets.BoundedFloatText(
 value=50,
 min=0,
 max=200,
 description='Radius:'
 )
wNumpoints = widgets.BoundedIntText(
 value=16,
 min=2,
 max=100,
 description='Num Points:'
 )
wNumaxes = widgets.BoundedIntText(
 value=8,
 min=2,
 max=20,
 description='Num Axes:'
 )
bCalculate = widgets.Button(
    description='Policz',
    icon='check' # (FontAwesome names without the `fa-` prefix),
)

In [6]:
#setup map
from ipyleaflet import Map, basemaps, basemap_to_tiles,LayersControl,ScaleControl,MeasureControl,\
  LayerGroup,Marker, Polygon 


mapnik = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
mapnik.base = True
mapnik.name = 'Mapnik Layer'

toner = basemap_to_tiles(basemaps.Stamen.Toner)
toner.base = True
toner.name = 'Toner Layer'

bzh = basemap_to_tiles(basemaps.OpenStreetMap.BZH)
bzh.base = True
bzh.name = 'BZH layer'

mapa =Map(center = (wLat.value, wLon.value), zoom = 17, min_zoom = 1, max_zoom = 20, 
    layers=[mapnik, toner, bzh])

measure = MeasureControl(
    position='bottomright',
    active_color = 'orange',
    primary_length_unit = 'meters',
    primary_area_unit = 'sqmeters',
    completed_color = 'red'
)
mapa.add_control(measure)
mapa.add_control(ScaleControl(position='bottomleft'))
mapa.add_control(LayersControl(position='topright'))

centerMarker = Marker(location=(wLat.value, wLon.value), draggable=True, name="Środek")
def centerMarker_on_location_changed(event):
  wLat.value, wLon.value = centerMarker.location

centerMarker.observe(centerMarker_on_location_changed, 'location')
mapa.add_layer(centerMarker)
pointsLayer = LayerGroup(name = 'Linie')
mapa.add_layer(pointsLayer)


In [7]:
#setup calculations
import math
from pyproj import Transformer, CRS, transform
def calculate():
  num_points = wNumpoints.value
  num_axes = wNumaxes.value
  lat = wLat.value
  lon = wLon.value
  radius = wRadius.value


  #Source projection
  sourceProj = CRS('epsg:4326')
  #Azimuthal Equidistant projection centred on your origin point
  localProj = CRS(proj="aeqd", lat_0=lat, lon_0=lon, datum="WGS84", units="m")
  transformer = Transformer.from_crs(sourceProj,localProj)
  invtransformer = Transformer.from_crs(localProj,sourceProj)
  
  x,y = transformer.transform(lat, lon)
  angle = 2 * math.pi / num_points

  pointsLocal = [( 
    x + (radius * math.cos(i * angle)),
    y + (radius * math.sin(i * angle))
    ) for i in range(num_points)]
  points = [*invtransformer.itransform(pointsLocal)]

  #angleAxes = 2 * math.pi / num_axes
  # pointMap = map(lambda i: [ 
  #         lat + (radius * math.cos(i * angle)),
  #         lon + (radius * math.sin(i * angle))]
  #         , range(wNumpoints.value))
  # points = list(pointMap)
  centerMarker.location =(lat,lon)
  mapa.center = (lat,lon)
  linia = Polygon(locations=points,
    color="green" ,
    fill=True)
  pointsLayer.clear_layers()
  pointsLayer.add_layer(linia) 
 
def bCalculate_on_click(e):
  calculate()
bCalculate.on_click(bCalculate_on_click)
  

In [8]:
#UI
display(wLat,wLon,wRadius,wNumpoints, wNumaxes,bCalculate)
display(mapa)

BoundedFloatText(value=50.936667, description='Lat:', max=60.0)

BoundedFloatText(value=16.861389, description='Lon:', max=20.0)

BoundedFloatText(value=50.0, description='Radius:', max=200.0)

BoundedIntText(value=16, description='Num Points:', min=2)

BoundedIntText(value=8, description='Num Axes:', max=20, min=2)

Button(description='Policz', icon='check', style=ButtonStyle())

Map(center=[50.936667, 16.861389], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…